Asynchrone Microservices
------------------------

![](images/Microservices-Messaging.png)

Source: Book Microservices Recipes
- - -

The system consists of a microservice **order** that accepts an order via the web interface.

The ordering process then sends the order as a record via Kafka to the microservice for shipping **shipping** and the microservice for creating the invoice **invoicing**.

The order is transmitted as JSON. In this way, the billing microservice and the shipping microservice can each read the data from the data structure that is relevant for the respective microservice.

The shipping microservice and the billing microservice store the information from the records in their own database schemas. All microservices use a common Postgres database.

- - -

First we create the ms-kafka Kubernetes namespace and label it so that Kiali can collect information.

In [ ]:
! kubectl create namespace ms-kafka
! kubectl label  namespace ms-kafka istio-injection=enabled

Now is a good time to launch the Kubernetes Dashboard and select the "ms-rest" namespace there.

Then we see how the next step is to create the microservices.

In [ ]:
%%bash
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/apache.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/invoicing.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/kafka.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/order.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/postgres.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/shipping.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kafka/zookeeper.yaml
kubectl get service/apache --namespace ms-kafka

Since we don't have a load balancer, we have to prepare the IP of the cluster and the mapped port (port-based-routing) as a URL using a small shell script.

If this URL is opened in a new tab, a menu is displayed.

In [ ]:
! echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-kafka apache -o=jsonpath='{ .spec.ports[0].nodePort }')

- -  -
After we have entered a few orders, we can look at the messages in the messaging server (Kafka).


In [ ]:
! kubectl exec -n ms-kafka  $(kubectl get po -l app=kafka -n ms-kafka -o=jsonpath='{ .items[0].metadata.name }') -- bash /opt/kafka/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic order --from-beginning --timeout-ms 2000 | jq 

---
## Kiali - Observability

Kiali from the Istio project helps to understand the structure (= service mesh) of the application and its microservices and interfaces.

In [ ]:
! kubectl get service -n istio-system -l app=kiali  -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! echo "Kiali   UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service -l app=kiali -o=jsonpath='{ .items[0].spec.ports[0].nodePort }')

- - -
## Jaeger - Tracing

Jaeger is a system to track calls between microservices.

In [ ]:
! kubectl get service -n istio-system -l app=jaeger -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! echo "Jaeger  UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service/tracing -o jsonpath='{.spec.ports[?(@.name=="http-query")].nodePort}')

- - -
## Lasttest

In order to make the connections visible, we generate a little traffic.

In [ ]:
%%bash
for i in `seq 1 1000`;
do
    curl -s -o /dev/null -I -w "%{http_code}" http://apache.ms-kafka/order/order
    echo
done

- - -

Clean up

In [ ]:
! kubectl delete namespace ms-kafka

- - -
## Grafana

Grafana is an open-source, cross-platform application for graphing data from various data sources such as B. InfluxDB, MySQL, PostgreSQL, Prometheus and Graphite. The recorded raw data can then be output in various display formats

In [ ]:
! kubectl get service -n istio-system -l app.kubernetes.io/instance=grafana -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! echo "Grafana  UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service -l app.kubernetes.io/instance=grafana -o=jsonpath='{ .items[0].spec.ports[0].nodePort }')                                                         